# 参数管理
- 访问参数，便于调试诊断
- 参数初始化
- 不同模型之间共享参数


In [ ]:
import torch 
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)


In [ ]:
import torch 
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(*[name for name, param in net[0].named_parameters()])
print(net[0].state_dict()) #第一个全连接层
print(net[1].state_dict()) #relu层
print(net[2].state_dict()) #第二个全连接层
print("------------------------------")
print(type(net[2].bias)) # 返回类型
print(net[2].bias)       # 打印bias
print(net[2].bias.data)
print("==============================")
print(*[(name,param.shape) for name,param in net.named_parameters()])

# 从嵌套块收集参数

In [ ]:
import torch
from torch import nn
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4, 1))
X=torch.rand(size=(2,4))
rgnet(X)
print(rgnet) #查看当前的网络结构
print(rgnet[0][1][0].bias.data) # 通过索引访问具体的值

# 参数初始化
好的初始化参数能够提升模型性能、增强泛化能力、提高计算效率

$ y=\sigma (wx+b) $

w-权重
b-偏置
$\sigma$-激活函数

In [ ]:
import torch
from torch import nn
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4, 1))
X=torch.rand(size=(2,4))
rgnet(X)
# print(rgnet) #查看当前的网络结构
# print(rgnet[0][1][0].bias.data) # 通过索引访问具体的值

def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.1)
        nn.init.zeros_(m.bias)  #这里初始化为零了

def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

def my_init(m):
    if type(m)==nn.Linear:
        print("init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5 # 创建一个等大小的布尔变量 要么为0，要么就是绝对值大于5



rgnet.apply(init_normal)
print(rgnet[0][1][0].weight.data)
print(rgnet[0][1][0].bias.data)

rgnet.apply(init_constant)      # 修改为常数
print(rgnet[0][1][0].weight.data)
print(rgnet[0][1][0].bias.data)

rgnet.apply(my_init)
print(rgnet[0][1][0].weight.data)
print(rgnet[0][1][0].bias.data)


# 参数共享
在多个层之间共享参数，可以定义一个稠密层，使用其参数来设置另一个层的采纳数，参数共享的好处有以下几点：
- 节省内存
- 对于图像识别可以全图的任何地方而不是只在特定的区域查找
- 对于RNN适应各个时间步骤
- 提高了泛化能力

In [ ]:
import torch
from torch import nn

shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))

X=torch.rand(size=(2,4))
print(net)
print(net(X))
print(net[2].weight.data==net[4].weight.data) # 判断是否相等